# CIFAR-10 CNN

**Convolution Neural Network**<br>
보통 이미지 데이터에 사용<br>
Convolutional Filter를 이미지에 적용해서 이미지로부터 feature 학습함

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_epochs = 4
batch_size = 4
learning_rate = 0.001

In [2]:
# Load Dataset
# CIFAR-10: PILImage images of range [0,1]
# Transform images to Tensor of normalized range [-1,1]
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [10]:
# Implement Convolutional Neural Net
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # Input -> [Conv -> ReLU -> Pooling]*n -> Flatten -> FC layer -> Softmax
        # input size: batch_size * 3 * 32*32
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)    # output size: 6*28*28
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)   # 2x2 Max Pool      # output size: 6*14*14
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)   # output size: 16*10*10 | 이후 pooling layer 거쳐야되기에 output size: 16*5*5
        self.fc1 = nn.Linear(in_features=16*5*5, out_features=120)  # in_features=16*5*5 고정 (이전에 들어오는 입력 크기가 16*5*5이기 때문)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)   # Class 10개라서 out_features=10 고정
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.relu(self.conv1(x))
        out = self.pool(out) 
        out = self.relu(self.conv2(out))
        out = self.pool(out)

        out = out.view(-1, 16*5*5)
        out = self.relu(self.fc1(out))
        out = self.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [16]:
model = ConvNet()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training Loop
num_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape [4,3,32,32] --> [4,3,1024]
        # Input layer: 3 Input channels     | 6 output channels |   5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if (i+1) % 2000 == 0:
            print(f'Epoch [{epoch+1} / {num_epochs}] | Step [{i+1} / {num_total_steps}] | Loss: {loss.item():.4f}')
print("Training Done!")

# Evaluation
with torch.no_grad():
    num_correct = 0
    num_samples = 0
    num_class_correct = [0 for i in range(10)]  # CIFAR-10: 10 classes
    num_class_samples = [0 for i in range(10)]

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _, predictions = torch.max(outputs, 1)
        num_samples += labels.size(0)
        num_correct += (predictions == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predictions[i]
            if (label == pred):
                num_class_correct[label] += 1
            num_class_samples[label] += 1
    acc = 100.0 * num_correct / num_samples
    print(f'Accuracy of CNN: {acc} %')

    for i in range(10):
        acc = 100.0 * num_class_correct[i] / num_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Epoch [1 / 4] | Step [2000 / 12500] | Loss: 2.3017
Epoch [1 / 4] | Step [4000 / 12500] | Loss: 2.2803
Epoch [1 / 4] | Step [6000 / 12500] | Loss: 2.2725
Epoch [1 / 4] | Step [8000 / 12500] | Loss: 2.2688
Epoch [1 / 4] | Step [10000 / 12500] | Loss: 2.2270
Epoch [1 / 4] | Step [12000 / 12500] | Loss: 2.3173
Epoch [2 / 4] | Step [2000 / 12500] | Loss: 1.4752
Epoch [2 / 4] | Step [4000 / 12500] | Loss: 2.5766
Epoch [2 / 4] | Step [6000 / 12500] | Loss: 2.0298
Epoch [2 / 4] | Step [8000 / 12500] | Loss: 1.9826
Epoch [2 / 4] | Step [10000 / 12500] | Loss: 1.8217
Epoch [2 / 4] | Step [12000 / 12500] | Loss: 2.1461
Epoch [3 / 4] | Step [2000 / 12500] | Loss: 1.3657
Epoch [3 / 4] | Step [4000 / 12500] | Loss: 2.0271
Epoch [3 / 4] | Step [6000 / 12500] | Loss: 2.0997
Epoch [3 / 4] | Step [8000 / 12500] | Loss: 1.2921
Epoch [3 / 4] | Step [10000 / 12500] | Loss: 1.6974
Epoch [3 / 4] | Step [12000 / 12500] | Loss: 1.5345
Epoch [4 / 4] | Step [2000 / 12500] | Loss: 1.5041
Epoch [4 / 4] | Step [400